# Data Analysis

## READ BEFORE DEVELOPING
If you make any changes to the files in `modules` you have to take the following steps to get those changes in here:
1. Shut down `jupyter notebook`
2. Run `pip install .` on project root directory
3. Start up `jupyter notebook` and re-run all of the cells (to make sure it gets the imports and all)

In [11]:
from modules import process as prc
from pkg_resources import resource_filename

In [12]:
raw_data = prc.read_file(resource_filename('data', 'training_data.json'))

In [13]:
# After loading in the raw data we want to clean out any instances were the was an invalid match
data = prc.filter_bad_data(raw_data)

In [14]:
# Get the hero win rate dataset
hero_winrates = prc.get_hero_winrates(data, resource_filename('data', 'heroes.json'))
print(hero_winrates)

{1: 45.047489823609226, 2: 51.47392290249433, 3: 47.3170731707317, 4: 48.643410852713174, 5: 51.02239532619279, 6: 51.77304964539007, 7: 47.48010610079575, 8: 49.032738095238095, 9: 51.93065405831363, 10: 44.9685534591195, 11: 48.33574529667149, 12: 52.40506329113924, 13: 39.58333333333333, 14: 52.54372019077902, 15: 49.09747292418773, 16: 50.0875656742557, 17: 47.682119205298015, 18: 49.2063492063492, 19: 47.1559633027523, 20: 51.12994350282486, 21: 53.00653594771242, 22: 54.095238095238095, 23: 48.2311320754717, 25: 46.31710362047441, 26: 50.05500550055005, 27: 52.26666666666666, 28: 49.91023339317774, 29: 52.45346869712352, 30: 51.29770992366412, 31: 52.35457063711911, 32: 59.118727050183594, 33: 49.80694980694981, 34: 45.84717607973422, 35: 43.976897689768975, 36: 47.18543046357616, 37: 50.74626865671642, 38: 42.64705882352941, 39: 44.35946462715105, 40: 50.728155339805824, 41: 53.437967115097166, 42: 54.665203073545555, 43: 42.30769230769231, 44: 51.045016077170416, 45: 48.4787018

In [15]:
# Get team win rates EXAMPLE
match = data[1192]['result']
teams = prc.determine_teams(match)
for name, team_roster in teams.items():
    team_winrate = prc.calculate_winrate_average(team_roster, hero_winrates)
    print(team_winrate)

50.35252314953314
49.51139202096101
